# Neural Networks

## 1. Import Libraries and Setup

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score, 
    roc_curve
)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from imblearn.over_sampling import SMOTE

# set random seeds
np.random.seed(42)
tf.random.set_seed(42)

## 2. Load Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('churn_data_cleaned.csv')

## 3. Prepare Train-Test Split

Split the data into training (80%) and testing (20%) sets with stratification to maintain class distribution.

In [ ]:
# Separate features and target
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Maintain class distribution
)

## 4. Feature Scaling

Neural networks require standardized features for optimal performance. We use StandardScaler to normalize all features.

In [ ]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 5. Define Model Architecture

Build a neural network with hyperparameters that will be tuned through cross-validation.

In [ ]:
def build_model(input_dim, learning_rate=0.0005, dropout_rate=0.4, l2_reg=0.001, hidden_size=256):
    """
    Build neural network model with tunable hyperparameters.
    """
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        
        # Hidden layer
        layers.Dense(hidden_size, activation='relu', 
                     kernel_regularizer=keras.regularizers.l2(l2_reg)),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        
        # Output layer
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )
    
    return model

## 6. Perform 5-Fold Cross-Validation

Use cross-validation to find the best hyperparameters. SMOTE is applied inside each fold to prevent data leakage.

In [ ]:
def cross_validate_nn(X, y, learning_rate, dropout_rate, l2_reg, batch_size, hidden_size, n_splits=5):
    """
    Perform 5-fold cross-validation with SMOTE applied inside each fold.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    fold_scores = {'precision': [], 'recall': [], 'auc': []}
    
    print(f"\nTesting: Hidden={hidden_size}, LR={learning_rate}, Dropout={dropout_rate}, L2={l2_reg}, Batch={batch_size}")
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"  Fold {fold}/{n_splits}...", end=" ")
        
        # Split data
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        # Apply SMOTE only to training fold
        smote = SMOTE(random_state=42, k_neighbors=5)
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train_fold, y_train_fold)
        
        # Build and train model
        model = build_model(X_train_fold.shape[1], learning_rate, dropout_rate, l2_reg, hidden_size)
        
        early_stop = callbacks.EarlyStopping(monitor='loss', patience=10, 
                                             restore_best_weights=True, verbose=0)
        
        model.fit(X_train_balanced, y_train_balanced,
                  epochs=50,
                  batch_size=batch_size,
                  callbacks=[early_stop],
                  verbose=0)
        
        # Evaluate on validation fold
        results = model.evaluate(X_val_fold, y_val_fold, verbose=0)
        
        # Store metrics (loss, accuracy, precision, recall, auc)
        fold_scores['precision'].append(results[2])
        fold_scores['recall'].append(results[3])
        fold_scores['auc'].append(results[4])
        
        print(f"Precision={results[2]:.4f}, Recall={results[3]:.4f}")
        
        # Clean up
        del model
        keras.backend.clear_session()
    
    # Calculate averages
    avg_scores = {metric: np.mean(scores) for metric, scores in fold_scores.items()}
    std_scores = {metric: np.std(scores) for metric, scores in fold_scores.items()}
    
    return avg_scores, std_scores

In [ ]:
# Hyperparameter grid search with 5-fold cross-validation
print("Parameter Grid")

hyperparameter_grid = [
    {'lr': 0.001, 'dropout': 0.3, 'l2': 0.001, 'batch': 64, 'hidden': 256},
    {'lr': 0.001, 'dropout': 0.4, 'l2': 0.001, 'batch': 64, 'hidden': 256},
    {'lr': 0.0005, 'dropout': 0.3, 'l2': 0.001, 'batch': 64, 'hidden': 256},
    {'lr': 0.001, 'dropout': 0.3, 'l2': 0.01, 'batch': 64, 'hidden': 256},
    {'lr': 0.001, 'dropout': 0.3, 'l2': 0.001, 'batch': 128, 'hidden': 256},
    {'lr': 0.001, 'dropout': 0.3, 'l2': 0.001, 'batch': 64, 'hidden': 512},
]

print(f"\nTotal combinations to test: {len(hyperparameter_grid)}")

results = []

for i, params in enumerate(hyperparameter_grid, 1):
    print(f"\n[Combination {i}/{len(hyperparameter_grid)}]")
    
    avg_scores, std_scores = cross_validate_nn(
        X_train_scaled, y_train,
        params['lr'], params['dropout'], params['l2'], params['batch'], params['hidden'],
        n_splits=5
    )
    
    results.append({
        'hidden_size': params['hidden'],
        'learning_rate': params['lr'],
        'dropout_rate': params['dropout'],
        'l2_reg': params['l2'],
        'batch_size': params['batch'],
        'avg_precision': avg_scores['precision'],
        'avg_recall': avg_scores['recall'],
        'avg_auc': avg_scores['auc'],
        'std_precision': std_scores['precision'],
        'std_recall': std_scores['recall']
    })
    
    print(f"  → Avg Precision: {avg_scores['precision']:.4f} ± {std_scores['precision']:.4f}")
    print(f"  → Avg Recall: {avg_scores['recall']:.4f} ± {std_scores['recall']:.4f}")

## 7. Display Cross-Validation Results

Show the best parameters found and the top parameter combinations based on precision score.

In [ ]:
# Display best parameters and cross-validation results
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('avg_precision', ascending=False)

print("\nBest Parameters Found:")
best_params = results_df.iloc[0]
print(f"  hidden_size: {int(best_params['hidden_size'])}")
print(f"  learning_rate: {best_params['learning_rate']}")
print(f"  dropout_rate: {best_params['dropout_rate']}")
print(f"  l2_reg: {best_params['l2_reg']}")
print(f"  batch_size: {int(best_params['batch_size'])}")

print(f"\nBest Cross-Validation Precision Score: {best_params['avg_precision']:.4f}")

print("\nTop 5 Parameter Combinations:")
for idx, row in results_df.head(5).iterrows():
    print(f"  Parameters: hidden={int(row['hidden_size'])}, lr={row['learning_rate']}, dropout={row['dropout_rate']}")
    print(f"  CV Precision Score: {row['avg_precision']:.4f} (+/- {row['std_precision']:.4f})")
    print(f"  CV Recall Score: {row['avg_recall']:.4f} (+/- {row['std_recall']:.4f})")
    print("\n")

## 8. Build and Train Final Model

Train the final model using the best hyperparameters found via cross-validation. SMOTE is applied to the full training set.

In [ ]:
# Apply SMOTE to full training set
smote_final = SMOTE(random_state=42, k_neighbors=5)
X_train_balanced, y_train_balanced = smote_final.fit_resample(X_train_scaled, y_train)

In [ ]:
# Build final model with best hyperparameters
model_final = build_model(
    input_dim=X_train_scaled.shape[1],
    learning_rate=best_params['learning_rate'],
    dropout_rate=best_params['dropout_rate'],
    l2_reg=best_params['l2_reg'],
    hidden_size=int(best_params['hidden_size'])
)

print(f"\nFinal model architecture:")
print(f"  Hidden layer size: {int(best_params['hidden_size'])} neurons")
print(f"  Learning rate: {best_params['learning_rate']}")
print(f"  Dropout rate: {best_params['dropout_rate']}")
print(f"  L2 regularization: {best_params['l2_reg']}")

model_final.summary()

In [ ]:
# Train final model
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

history = model_final.fit(
    X_train_balanced, y_train_balanced,
    epochs=100,
    batch_size=int(best_params['batch_size']),
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

## 9. Evaluate Model Performance

Make predictions on the test set and calculate performance metrics.

In [ ]:
# Make predictions on test set
y_pred_proba = model_final.predict(X_test_scaled).flatten()
y_pred = (y_pred_proba >= 0.5).astype(int)

# Calculate metrics
print("\nNeural Network Classification Report:")
print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

# Calculate individual metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)

print("Performance Metrics")
print(f"Precision:  {precision:.4f}")
print(f"Recall:     {recall:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"AUC-ROC:    {auc_roc:.4f}")

## 10. Visualize Results

Plot confusion matrix and ROC curve to visualize model performance.

In [ ]:
# Plot confusion matrix and ROC curve
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', ax=axes[0])
axes[0].set_title('Confusion Matrix', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_xticklabels(['No Churn', 'Churn'])
axes[0].set_yticklabels(['No Churn', 'Churn'])

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
axes[1].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc_roc:.4f})', color='red')
axes[1].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
axes[1].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate (Recall)')
axes[1].legend(loc='lower right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()